In [1]:
import pandas as pd
import numpy as np
import pathlib as pl
import sklearn
import pathlib as pl

In [2]:
dictionarypath = '../dataset/dictionary/Loughran-McDonald_MasterDictionary_1993-2021.csv'
wds_dict = pd.read_csv(dictionarypath)
positive_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Positive']!=0]
negative_wds = [row['Word'].lower() for i, row in wds_dict.iterrows() if row['Negative']!=0]

In [3]:
path = '../dataset/finRep/FinancialPhraseBank/all-data.csv'
df = pd.read_csv(path, encoding = "ISO-8859-1", names = ['sentiment', 'sentence'])
# print(df[:10])

In [14]:
# to install keybert
# !pip install nltk
# !pip install keybert
# !pip install sentence-transformers
# !pip install git+https://github.com/LIAAD/yake
### To show the execution times
# !pip install ipython-autotime
# %load_ext autotime

In [58]:
# choose a random sample
text = df.sample(1)['sentence'].iloc[0]
label = df.sample(1)['sentiment'].iloc[0]
print(text)
print("the sentiment is : %s"%label)

It will also strengthen Ruukki 's offshore business .
the sentiment is : positive


In [59]:
def normalize_keyword_list(keyword_list):
    keys = []
    for keyword in keyword_list:
        keys.append(keyword[0])
    return keys

Rake

In [60]:
from rake_nltk import Rake
rake_nltk_var = Rake()

rake_nltk_var.extract_keywords_from_text(text)
keyword_extracted = rake_nltk_var.get_ranked_phrases()
print(keyword_extracted)

['also strengthen ruukki', 'offshore business']


In [61]:
# Import them
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
sent_trans = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
keyBERT_model = KeyBERT(model = sent_trans)

In [62]:
def extract_terms(document, n_gram_range = (3,3), 
                  top_N=5, model=keyBERT_model, diversity_threshold = 0.7):

  keywords = model.extract_keywords(document, stop_words='english', 
                                    keyphrase_ngram_range=(1, 3),
                                    use_mmr=True, 
                                    diversity = diversity_threshold,
                                    top_n = top_N)
  
  return sorted(keywords, key=lambda tup:(-tup[1], tup[0]))

In [63]:
print(f'Text: {text}.\nLength: {len(text.split())}')
best_terms = extract_terms(text)
best_terms = normalize_keyword_list(best_terms)

print(best_terms)

Text: It will also strengthen Ruukki 's offshore business ..
Length: 9
['ruukki offshore business', 'strengthen ruukki', 'offshore', 'strengthen', 'business']


In [64]:
import yake
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(text)
keywords = normalize_keyword_list(keywords)

print(keywords)


# print("SECOND FORM:")
# language = "en"
# max_ngram_size = 3
# deduplication_threshold = 0.9
# deduplication_algo = 'seqm'
# windowSize = 1
# numOfKeywords = 20

# custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
# keywords = custom_kw_extractor.extract_keywords(text)

# for kw in keywords:
#     print(kw)

['strengthen Ruukki', 'offshore business', 'Ruukki', 'business', 'strengthen', 'offshore']


NLTK-VADER

In [68]:
print(best_terms)

['ruukki offshore business', 'strengthen ruukki', 'offshore', 'strengthen', 'business']


In [69]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

result = []
for sentence in best_terms:
    s = sentence.split()
    keyword_sen = []
    for word in s:
        if (sid.polarity_scores(word)['compound']) >= 0.5:
            keyword_sen.append(tuple((word, 1)))
        elif (sid.polarity_scores(word)['compound']) <= -0.5:
            keyword_sen.append(tuple((word, -1)))
        else:
            keyword_sen.append(tuple((word, 0)))
    result.append(keyword_sen)

print(result)              

[[('ruukki', 0), ('offshore', 0), ('business', 0)], [('strengthen', 0), ('ruukki', 0)], [('offshore', 0)], [('strengthen', 0)], [('business', 0)]]


Using TextBlob

In [71]:
from textblob import TextBlob

result = []
for sentence in best_terms:
    s = sentence.split()
    keyword_sen = []
    for word in s:
        testimonial = TextBlob(word)
        if testimonial.sentiment.polarity >= 0.5:
            keyword_sen.append(tuple((word, 1)))
        elif testimonial.sentiment.polarity <= -0.5:
            keyword_sen.append(tuple((word, -1)))
        else:
            keyword_sen.append(tuple((word, 0)))
    result.append(keyword_sen)
print(result)


[[('ruukki', 0), ('offshore', 0), ('business', 0)], [('strengthen', 0), ('ruukki', 0)], [('offshore', 0)], [('strengthen', 0)], [('business', 0)]]


Using dictionary

In [65]:
def compare_words(keyword_list):
    result = []
    for sentence in keyword_list:
        words = sentence.split()
        keyword_sen = []
        for word in words:
            if word in positive_wds:
                keyword_sen.append(tuple((word, 1)))
            elif word in negative_wds:
                keyword_sen.append(tuple((word, -1)))
            else:
                keyword_sen.append(tuple((word, 0)))
        result.append(keyword_sen)
    return result

In [66]:
keybert_sen = compare_words(best_terms) #keybert

print(keybert_sen)

[[('ruukki', 0), ('offshore', 0), ('business', 0)], [('strengthen', 1), ('ruukki', 0)], [('offshore', 0)], [('strengthen', 1)], [('business', 0)]]


Using Flair

In [76]:
# !pip install flair

  Using cached flair-0.11.3-py3-none-any.whl (401 kB)
  Using cached conllu-4.5.2-py2.py3-none-any.whl (16 kB)
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached gdown-4.4.0-py3-none-any.whl
  Using cached Wikipedia-API-0.5.4.tar.gz (18 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached matplotlib-3.5.3-cp37-cp37m-win_amd64.whl (7.2 MB)
  Using cached mpld3-0.3.tar.gz (788 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cached pptree-3.1.tar.gz (3.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using 


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached flair-0.11.3-py3-none-any.whl (401 kB)
  Using cached konoha-4.6.5-py3-none-any.whl (20 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-win_amd64.whl (7.2 MB)
  Using cached gensim-4.2.0-cp37-cp37m-win_amd64.whl (24.0 MB)
  Using cached bpemb-0.3.4-py3-none-any.whl (19 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached conllu-4.5.2-py2.py3-none-any.whl (16 kB)
  Using cached lxml-4.9.1-cp37-cp37m-win_amd64.whl (3.6 MB)
  Using cached more_itertools-8.14.0-py3-none-any.whl (52 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached gdown-4.4.0-py3-none-any.whl
  Using cached Wikipedia-API-0.5.4.tar.gz (18 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cache


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [84]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('en-sentiment')

result = []
for sentence in best_terms:
    s = sentence.split()
    keyword_sen = []
    for word in s:
        word = Sentence(word)
        classifier.predict(word)
        if (word.labels[0].score) >= 0.5:
            keyword_sen.append(tuple((word, 1)))
        elif (word.labels[0].score) <= -0.5:
            keyword_sen.append(tuple((word, -1)))
        else:
            keyword_sen.append(tuple((word, 0)))
    result.append(keyword_sen)

print(result) 

2022-10-16 18:49:44,443 loading file C:\Users\Rolando\.flair\models\sentiment-en-mix-distillbert_4.pt
[[(Sentence: "ruukki" → NEGATIVE (0.9388), 1), (Sentence: "offshore" → POSITIVE (0.9857), 1), (Sentence: "business" → POSITIVE (0.9777), 1)], [(Sentence: "strengthen" → POSITIVE (0.9987), 1), (Sentence: "ruukki" → NEGATIVE (0.9388), 1)], [(Sentence: "offshore" → POSITIVE (0.9857), 1)], [(Sentence: "strengthen" → POSITIVE (0.9987), 1)], [(Sentence: "business" → POSITIVE (0.9777), 1)]]
